# Standard packages

In [4]:
import os
import pandas as pd
import time

# Specific installs

pKiss installed through Bioconda.

**Note:** if the colab session fails (with unkown reason), run all the cells again. It should work ok the second time. 

In [5]:
method_name = "pKiss"

In [6]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c bioconda pkiss

✨🍰✨ Everything looks OK!
Solving environment: - \ | / - \ | / - \ | / - \ | / done

# All requested packages already installed.



Now needs FoldGrammars sourcess

# S. cerevisiae (sce) 18 long non-coding RNA dataset
Data source: https://genie.weizmann.ac.il/pubs/PARS10/data/sce_genes_folded.tab.gz

In [7]:
gh_path = "https://raw.githubusercontent.com/sinc-lab/lncRNA-folding/master/data/"
sce = pd.read_csv(gh_path + "sce_genes_folded.tab", delimiter='\t', 
                  header=None, index_col=0, 
                  names=("Gene ID", "sequence", "PARS-assisted folding"))

In [8]:
# Sequences to process
yeast18lnc = ["snR81", "snR34", "snR43", "snR44",  "snR31",  "snR10",
              "snR63", "snR11", "snR82", "snR17b", "snR17a", "snR37",
              "SCR1",  "SRG1",  "snR19", "snR30",  "LSR1",   "TLC1"]

# Compute structures

In [9]:
def run_folding(fasta_name):

  # Compute structure
  os.system("pKiss -mode mfe --temperature 25 tmp.fasta > pkiss_tmp.fasta")
    
  # Clean output
  os.system("cat pkiss_tmp.fasta | sed '/^$/d' | sed 's/^ \+//g' | sed 's/ \+/\t/g' | cut -f2 > pkiss_tmp_cleaned.fasta")
  return "pkiss_tmp_cleaned.fasta"

In [10]:
out_fasta_name = method_name + "_yeast18"
if os.path.exists(out_fasta_name + ".fasta"): os.remove(out_fasta_name + ".fasta")

lnc_ids = yeast18lnc
print("   \t lnc \t len \t time")
for i, lnc in enumerate(lnc_ids): 

  start_time = time.time()
  seq = sce.loc[lnc]["sequence"]
  print(f"{i+1}/{len(lnc_ids)}\t{lnc} \t {len(seq)}", end='\t')

  # Write a one-sequence fasta
  with open("tmp.fasta", "w") as ofile: 
    ofile.write(f">{lnc}\n{seq}\n")
  
  dot_file_name = run_folding("tmp.fasta")

  # Concatenate outputs
  os.system("cat " + dot_file_name + " >> " + out_fasta_name + ".fasta") 

  print(f"{time.time() - start_time: .1f} s")

   	 lnc 	 len 	 time
1/18	snR81 	 201	 2.9 s
2/18	snR34 	 203	 4.2 s
3/18	snR43 	 209	 3.5 s
4/18	snR44 	 211	 3.5 s
5/18	snR31 	 225	 4.7 s
6/18	snR10 	 245	 6.5 s
7/18	snR63 	 255	 8.1 s
8/18	snR11 	 258	 8.5 s
9/18	snR82 	 268	 9.4 s
10/18	snR17b 	 332	 22.1 s
11/18	snR17a 	 333	 21.3 s
12/18	snR37 	 386	 40.9 s
13/18	SCR1 	 522	 143.7 s
14/18	SRG1 	 551	 149.3 s
15/18	snR19 	 568	 212.1 s
16/18	snR30 	 606	 253.4 s
17/18	LSR1 	 1175	 4408.9 s
18/18	TLC1 	 1301	 5423.6 s
